In [ ]:
!pip install transformers datasets evaluate huggingface_hub wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sp4ss (sp4ss-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
#@Title: Training
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
dataset = load_dataset("DanielHesslow/SwissProt-Pfam")
train_ds = dataset["train"].train_test_split(test_size=0.1, seed=42)
val_ds = train_ds["test"]
train_ds = train_ds["train"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/494 [00:00<?, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/149M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/409267 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50530 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/51557 [00:00<?, ? examples/s]

In [ ]:
dataset ,val_ds, train_ds

(DatasetDict({
     train: Dataset({
         features: ['seq', 'labels', 'labels_str', 'id'],
         num_rows: 409267
     })
     test: Dataset({
         features: ['seq', 'labels', 'labels_str', 'id'],
         num_rows: 50530
     })
     dev: Dataset({
         features: ['seq', 'labels', 'labels_str', 'id'],
         num_rows: 51557
     })
 }),
 Dataset({
     features: ['seq', 'labels', 'labels_str', 'id'],
     num_rows: 40927
 }),
 Dataset({
     features: ['seq', 'labels', 'labels_str', 'id'],
     num_rows: 368340
 }))

In [ ]:
unique_labels = set(train_ds["labels_str"])
num_labels = len(unique_labels)
label2id = {label: i for i, label in enumerate(sorted(unique_labels))}
id2label = {i: label for label, i in label2id.items()}

In [64]:
# # 2. Setup tokenizer + model
# model_name = "facebook/esm2_t6_8M_UR50D"
# tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
# num_labels = len(set(train_ds["pfam_accession"]))
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name, num_labels=num_labels
# )
# model_name = "facebook/esm2_t6_8M_UR50D"
# tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name, num_labels=num_labels,
#     id2label=id2label, label2id=label2id
# )

from transformers import AutoModelForSequenceClassification
model="lhlkh"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification",
    id2label=id2label,
    label2id=label2id,
)

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
# 4. Preprocessing
# def preprocess(batch):
#     tokens = tokenizer(batch["seq"], truncation=True, padding="max_length", max_length=512)
#     tokens["labels"] = [label2id[label] for label in batch["labels_str"]]
#     return tokens


# Rebuild label2id from all unique labels across the whole dataset
from itertools import chain

all_labels = list(set(chain(*train_ds["labels_str"])))
label2id = {label: i for i, label in enumerate(sorted(all_labels))}
id2label = {i: label for label, i in label2id.items()}
num_labels = len(label2id)



# Updated preprocess for multi-label
# def preprocess(batch):
#     tokens = tokenizer(batch["seq"], truncation=True, padding="max_length", max_length=512)

#     # Convert each list of labels to a multi-hot vector
#     labels = []
#     for label_list in batch["labels_str"]:
#         label_vec = [0] * num_labels
#         for label in label_list:
#             if label in label2id:
#                 label_vec[label2id[label]] = 1
#         labels.append(label_vec)

#     tokens["labels"] = labels
#     return tokens
# def preprocess(batch):
#     tokens = tokenizer(batch["seq"], truncation=True, padding="max_length", max_length=512)

#     # Convert multi-label list to multi-hot float vector
#     labels = []
#     for label_list in batch["labels_str"]:
#         label_vec = [0.0] * num_labels  # note the float values
#         for label in label_list:
#             if label in label2id:
#                 label_vec[label2id[label]] = 1.0
#         labels.append(label_vec)

#     tokens["labels"] = labels
#     return tokens

def preprocess(batch):
    tokens = tokenizer(batch["seq"], truncation=True, padding="max_length", max_length=512)
    labels = []
    for label_list in batch["labels_str"]:
        label_vec = [float(0.0)] * num_labels  # explicit float zero
        for label in label_list:
            if label in label2id:
                label_vec[label2id[label]] = float(1.0)  # explicit float one
        tokens["labels"] = labels
        labels.append(label_vec)
    return tokens

In [ ]:
from itertools import chain

all_labels = list(set(chain(*train_ds["labels_str"])))
label2id = {label: i for i, label in enumerate(sorted(all_labels))}
id2label = {i: label for label, i in label2id.items()}
num_labels = len(label2id)

In [46]:
def preprocess(batch):
    tokens = tokenizer(
        batch["seq"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    labels = []
    for label_list in batch["labels_str"]:
        label_vec = [0.0] * num_labels  # float zeros
        for label in label_list:
            if label in label2id:
                label_vec[label2id[label]] = 1.0
        labels.append(label_vec)
    tokens["labels"] = labels
    return tokens

In [48]:
# train_tok = train_ds.map(preprocess, batched=True)
# val_tok = val_ds.map(preprocess, batched=True)
import torch
train_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"], dtype={"labels": torch.float})
val_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"], dtype={"labels": torch.float})

In [53]:
import datasets
train_tok = train_tok.cast_column("labels", datasets.Sequence(datasets.Value("float32")))
val_tok   = val_tok.cast_column("labels", datasets.Sequence(datasets.Value("float32")))

train_tok.set_format("torch")
val_tok.set_format("torch")

Casting the dataset:   0%|          | 0/368340 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/40927 [00:00<?, ? examples/s]

Map:   0%|          | 0/40927 [00:00<?, ? examples/s]

In [54]:
print(type(train_tok[0]['labels']), type(val_tok[0]['labels'][0]))
len(train_tok), len(val_tok)

<class 'torch.Tensor'> <class 'torch.Tensor'>


(368340, 40927)

In [32]:
training_args = TrainingArguments(
    output_dir="./pfam-clf",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    report_to="wandb",
)

In [31]:
# 6. Metrics function
# def compute_metrics(p):
#     preds = np.argmax(p.predictions, axis=1)
#     return {
#         "accuracy": accuracy_score(p.label_ids, preds),
#         "f1": f1_score(p.label_ids, preds, average="weighted")
#     }

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    labels = labels.astype(int)

    return {
        "f1_weighted": f1_score(labels, preds, average="weighted", zero_division=0),
        "f1_micro": f1_score(labels, preds, average="micro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }


In [55]:
# 7. Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics,
)

In [56]:
# 8. Train
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
#@title  Train 5
training_args = TrainingArguments(
    output_dir="./pfam-clf",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=48,
    per_device_eval_batch_size=64,
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="eval_accuracy",
    metric_for_best_model="f1_micro",
    report_to="wandb",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss


In [69]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# hf

In [ ]:
model.push_to_hub("Pingsz/mini-fold-classifier-esm2", use_auth_token=True)
tokenizer.push_to_hub("Pingsz/mini-fold-classifier-esm2", use_auth_token=True)

In [ ]:
# 10. Inference example
from transformers import pipeline
classifier = pipeline("text-classification", model="Pingsz/mini-fold-classifier-esm2")
print(classifier("MKTFFLILALLFSAYAGDTVNSVSLTSDVK..."))

In [ ]:
# Install the Transformers library
!pip install -q transformers

# Import pipeline from Hugging Face Transformers
from transformers import pipeline

# Load the model (this will download the model if not already available)
classifier = pipeline("text-classification", model="Pingsz/mini-fold-classifier-esm2")

# Sample test protein sequences for evaluation
# Create a Dataset and upload to huggingface
test_sequences = [
    "MKTFFLILALLFSAYAGDTVNSVSLTSDVK",
    "MAEPRQEFEVMVVRVDPRLCVILKRRPVEK",
    "MGSSHHHHHHSSGLVPRGSHMGSH",
    "MKADRLKAVVHVNSGQG",  # Shorter sequence
    "MGLSDGEWQLVLNVWGKVEADIPGHGQEVL"  # Hemoglobin fragment
]

# Run inference on each sequence and print results
for seq in test_sequences:
    result = classifier(seq)
    print(f"Sequence: {seq}\nPrediction: {result}\n")
